In [ ]:
import os
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import scipy.cluster.hierarchy as spc

pio.templates.default = "plotly_white"

In [ ]:
try:
    _ = first_run
except NameError:
    first_run = True
    os.chdir(os.getcwd().rsplit("/", 1)[0])
    from _aux import functions as func

# Load data

In [ ]:
default = (
    pd.read_csv(
        "../data/train/X_train.csv",
        index_col=0,
        usecols=[
            "row_id",
            "account_amount_added_12_24m",
            "sum_capital_paid_account_0_12m",
            "sum_capital_paid_account_12_24m",
            "account_incoming_debt_vs_paid_0_24m",
            "num_unpaid_bills",
            "sum_paid_inv_0_12m",
            "max_paid_inv_0_12m",
            "max_paid_inv_0_24m",
            "num_active_inv",
            "num_active_div_by_paid_inv_0_12m",
        ],
    )
    .join(pd.read_csv("../data/train/y_train.csv", index_col=0))
    .query("default == 1")
)

not_default = (
    pd.read_csv(
        "../data/train/X_train.csv",
        index_col=0,
        usecols=[
            "row_id",
            "account_amount_added_12_24m",
            "sum_capital_paid_account_0_12m",
            "sum_capital_paid_account_12_24m",
            "account_incoming_debt_vs_paid_0_24m",
            "num_unpaid_bills",
            "sum_paid_inv_0_12m",
            "max_paid_inv_0_12m",
            "max_paid_inv_0_24m",
            "num_active_inv",
            "num_active_div_by_paid_inv_0_12m",
        ],
    )
    .join(pd.read_csv("../data/train/y_train.csv", index_col=0))
    .query("default == 0")
)

df = pd.read_csv(
    "../data/train/X_train.csv",
    index_col=0,
).join(pd.read_csv("../data/train/y_train.csv", index_col=0))

## Correlation between "account" variables

As shown in our sanity profile report, account variables have high correlation amongst themselves mainly due to overlaping lookback windows for aggregation. The first thing we must do is choose one (or some) of them to represent the group, which can be achieved by correlation clustering.

In [ ]:
corr = pd.concat([default, not_default]).corr()
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

fig = go.Figure()

fig.add_trace(
    go.Heatmap(
        z=corr.mask(mask),
        x=corr.columns,
        y=corr.columns,
        colorscale=px.colors.diverging.RdBu,
        zmin=-1,
        zmax=1,
    )
)

fig.update_layout(
    title="Correlation between 'account' variables",
    yaxis_autorange="reversed",
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    width=1000,
    height=500,
)

# fig.update_traces(opacity=0.6)
fig.show()

In [ ]:
pdist = spc.distance.pdist(corr)
linkage = spc.linkage(pdist, method="single")
idx = spc.fcluster(linkage, 0.5 * pdist.max(), "distance")

columns = [default.columns.tolist()[i] for i in list((np.argsort(idx)))]
clusterd_corr = pd.concat([default, not_default]).reindex(columns, axis=1).corr()

mask = np.zeros_like(clusterd_corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

corr["default"].to_frame(name="corr_with_label").assign(cluster=idx).drop(
    "default"
).sort_values(["cluster", "corr_with_label"], ascending=[True, False])

Despite our knowing that correlation is not equal to causation, the lack of proper time for exploration leads us to the not-so-ideal path of using correlation to exclude variables. At this time. we choose to analyze only the variable "num_active_div_by_paid_inv_0_12m" at the cost of probably losing some good insights.

## 1. "num_active_div_by_paid_inv_0_12m"

This variable has 22.8% of missing values and 48.7% of zero values, on top of some hard-hitting outliers.
One behaviour this variable describes is whether the customer has had a good experience with us and has come back for more. On the flip side, users that have not finished paying their first invoice yet (in a 12 month span) are attributed NaN simple because division by zero is undefined. Hence, a good customer will get NaN on their first purchase, but also an ill-intentioned customer that executes multiple orders with no intention to pay them. Therefore, we choose to fill missing values with the number of active invoices so that the latter can become visible through this variable.

Next, we look at how defaults are distributed across the categories.


In [ ]:
df[["num_active_div_by_paid_inv_0_12m"]].assign(
    num_active_div_by_paid_inv_0_12m=df[
        "num_active_div_by_paid_inv_0_12m"
    ].combine_first(df["num_active_inv"])
).describe(np.append(np.arange(0.25, 1.0, 0.1), np.array([0.99])))

In [ ]:
_, var_profile = func.test_k_prop(
    df[["default"]]
    .assign(
        var=df["num_active_div_by_paid_inv_0_12m"].combine_first(df["num_active_inv"]),
        bins_var=lambda frame: pd.cut(frame["var"], range(-1, 10)),
    )
    .groupby("bins_var")
    .agg(
        default=("default", "sum"),
        not_default=("default", func.complement),
        count=("default", "count"),
    )
    .transform(lambda s: s.astype(int))
)

var_profile

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=var_profile.reset_index().index.to_list(),
        y=(var_profile["not_default"] / var_profile["count"]),
        name="not_default",
    )
)

fig.add_trace(
    go.Bar(
        x=var_profile.reset_index().index.to_list(),
        y=(var_profile["default"] / var_profile["count"]),
        name="default",
    )
)

fig.update_layout(
    title="Distribution of observations across 'num_active_div_by_paid_inv_0_12m' classes",
    barmode="relative",
    yaxis_title="Percentage",
    yaxis_tickformat="%",
    xaxis_title="Ordered Bins",
    xaxis_tickvals=[x for x in range(0, var_profile.shape[0])],
)

fig.update_traces(opacity=0.75)
fig.show()

Let's take a look at the impact of our imputing by printing the number of new cases being considered for each category

In [ ]:
(
    df[["num_active_div_by_paid_inv_0_12m", "default"]]
    .assign(
        # num_active_div_by_paid_inv_0_12m=df["num_active_div_by_paid_inv_0_12m"].combine_first(df["num_active_inv"]),
        bins_var=pd.cut(df["num_active_div_by_paid_inv_0_12m"], range(-1, 10))
    )
    .groupby("bins_var")
    .agg(
        default=("default", "sum"),
        not_default=("default", func.complement),
        count=("default", "count"),
    )
    .transform(lambda s: s.astype(int))
    .assign(
        default=lambda frame: var_profile["default"] - frame["default"],
        not_default=lambda frame: var_profile["not_default"] - frame["not_default"],
        count=lambda frame: var_profile["count"] - frame["count"],
    )
)

We see that most new observations due to imputing fall in the first bin (equals zero). However, percentagewise, the increase of default cases in categories that hold less observations is significantly higher, which indicates that our imputing strategy was indeed a good idea.

Now, let's look at the impact of transforming this variable into a binary one.

In [ ]:
_, bool_var_profile = func.test_k_prop(
    df[["default"]]
    .assign(
        var=df["num_active_div_by_paid_inv_0_12m"].combine_first(df["num_active_inv"]),
        bins_var=lambda frame: pd.cut(frame["var"], [-1, 1, np.inf]),
    )
    .groupby("bins_var")
    .agg(
        default=("default", "sum"),
        not_default=("default", func.complement),
        count=("default", "count"),
    )
    .transform(lambda s: s.astype(int))
)

bool_var_profile

It is interesting that the contamination rate in the "bellow 1" class is very close to the dataset as a whole (~0.014), but the "above 1" class gets a nearly 10-fold increase in its contamination (~.13). It seems that transforming this variable into boolean will still capture some of the "default" behaviour whilst reducing the trouble with the Curse of Dimensionality.

How does this new variable correlate to the target label?

In [ ]:
df[["default"]].assign(
    var=df["num_active_div_by_paid_inv_0_12m"].combine_first(df["num_active_inv"]),
    above_1=lambda frame: (frame["var"] > 1).astype(float),
).corr()["default"].to_frame(name="corr_with_label").drop("default")

It is to be expected that reducing granularity will also reduce power. However, we must remind ourselves that correlation is not causation, and that the trade-off should be worth it. In conclusion, we decide to move forward with the boolean (above 1?) equivalent of the "num_active_div_by_paid_inv_0_12m" variable.

Next, we look at "archived" variables.